# Cuantificación de Madera Anormal en discos de pino

# 1  Descripción del Problema



Durante su crecimiento el pino puede desarrollar madera anormal. Este tipo de madera afecta las propiedades físicas, químicas y mecánicas de la madera con respecto a una madera “normal”. En la Figura 1.1 se observa el corte de un disco de pino mientras que en la Figura 1.2 se amplifica una región con madera anormal.

|                 |                   |
|:-----------------------------:|:-----------------------------:|
| <img src="entregable/troncoPreZoom.png"  > |<img src="entregable/zoom.png" style="width:500px;height:550px;">   | 
                               
Figura 1.1/1 Imagenes de disco de tronco original y amplificada.

Es de interes detectar este tipo de madera para disminuír costos económicos en la cadena de producción. Por ejemplo, al aserrar las tablas que se obtienen del tronco del pino, las mismas se “tuercen”, generándose perdidas de material. Además, los distintos tipos de madera, con diferencias en sus propiedades, producen que la madera se seca a distintos niveles de temperatura y humedad generándose contracciones. Induciendose rajado de las laminas.

El objetivo de este trabajo es implementar un algoritmo semiautomático que permite detectar la presencia de madera de comprensión en el corte de disco de pino.

# 2 Algoritmo propuesto



El método propuesto consiste en determinar si se está ante la presencia de madera de compresión en las 8 direcciones radiales principales (puntos cardinales) de la foto del tronco. Opcionalmente también es posible indicar otra dirección. 
<p>
    <img src='./entregable/ptos.png'  alt>
</p>

Para esto se emplea el algoritmo de la Figura 2.1. En ésta se tiene el diagrama de bloques del algoritmo propuesto. Cómo se observa el mismo está compuesto por 4 bloques principales. **Cómo parámetro de entrada se requiere las coordenadas del centro del disco.** 
<p>
    <img src='./entregable/algoritmo.png'  style="width:700px;height:500px;">
    <center>Figura 2.1 Diagrama de bloques algortimo propuesto</center>    
</p>


## 2.1 Detección de rajaduras 

El primer bloque consiste en determinar si se tiene una rajadura en alguna de las 8 direcciones principales (octantes). En caso de que se tenga una rajadura en algunos de los octantes, no sé realiza la clasificación del mismo (si hay o no madera de compresión). 

Se trabaja con la imagen en escala de grises. Además, en este bloque se realiza la segmentación de la imagen. La métodología que se utiliza para segmentar la imagen es la siguiente: 

Primero se aplica un filtro de mediana utilizando una ventana circular de radio 5 pixeles para suavizar la imagen. Luego se utiliza el algoritmo de Canny para determinar los bordes de toda la imagen. Se puede observar en la Figura 2.2 el resultado.

<p>
    <img src='./entregable/canny.png' style="width:400px;height:400px;" >
    <center>Figura 2.2 Detección de bordes con Canny </center>    
</p>

Una vez que se tienen determinados los bordes, se utilizan operaciones morfólogicas (dilatación, erosión y llenado de huecos) para realizar la segmentación. El resultado se observa en la Figura 2.4. Se utiliza la imagen segmentada ya que facilita la detección de rajaduras (mejores resultados). Además facilita enormemente la extracción automática de los perfiles si el fondo es uniforme. 

|                 |                   |
|:-----------------------------:|:-----------------------------:|
| <img src="entregable/troncoSin.png"  style="width:800px;height:450px;"> |<img src="entregable/troncoSegmentado.png">   | 
                               
Figura 2.3/4 Imagenes de disco de tronco original y segmentada.

Luego de segmentar la imagen se extraen los octantes para su posterior clasificación. El procedimiento para extraer los octantes consiste en lo siguiente: Con el fin de aplicar una transformación geométrica de rotación sobre el disco se aplica una traslación del centro del disco hacía el centro de la imagen. Luego se construye una ventana rectangular en el octante norte utilizando la ubicación del centro de la imagen (centro del disco). Está ventana se observan en la Figura 2.5.

<p>
    <img src='./entregable/extraccionOctante2.png'  style="width:300px;height:300px;">
    <center>Figura 2.5 Extracción de Octante </center>    
</p>


Se extrae el primer octante (Norte). A continuación se aplica una rotación central de 45 grados. De está manera se tiene que la ventana está recuadrando el octante Noreste. Se procede nuevamente a extraer el octante y continuar con la rotación central hasta extraer los 8 octantes. 


Luego de tener los octantes, se compara utilizando descriptores ORB contra una imagen con rajadura y otra sin. La imagen contra la que se tenga más matches es la que determina la clasificación del octante.  

|                 |                   |
|:-----------------------------:|:-----------------------------:|
| <img src="entregable/raja.png" > |<img src="entregable/sinRaja.png" style="width:100px;height:280px;">   | 
Figura 2.6/7 Imagenes de disco de tronco con y sin rajadura.

## 2.2 Extracción de perfiles

Se utilizan dos algoritmos diferentes para la extracción de perfiles dependiendo si lo que se desea es extraer los 8 radios principales ó un radio (segmento) a elección. Para extraer un radio se necesitan dos puntos. En el caso de los radios principales se tiene que uno de los extremos es el centro (el cuál debe ser ingresado por el usuario) y el otro es el que se encuentra en el borde del tronco a determinado ángulo (dependiendo el perfil de octante que se desee extraer). Comó la imagen está segmentada se puede asumir qué los únicos pixeles con valor de backgroud negro son los  del fondo. De esa manera, es relativamente sencillo (se cuenta con punto de inicio, dirección y condición de parada) determinar cuáles son las coordenadas de los pixeles del perfil que se desea obtener.


Por otro lado, cuándo lo que se desea es extraer el perfil dado por dos puntos indicados por el usuario, se extrae el perfil del segmento determinado por ambos puntos.


<p>
    <img src='./entregable/perfil.png'  alt>
    <center>Figura 2.8 Perfil de dirección norte del disco F10A. El indice 0 refiere al centro del tronco </center>    
</p>

## 2.3 Determinar radios acumulados

Dado que es de interes determinar si se tiene madera de compresión en el crecimiento del tronco anual es necesario determinar que partes del perfil corresponden a cada año de crecimiento. Se brindan dos opciones para determinar estos radios. La más precisa requiere que el usuario lo determine marcando los mismos de manera manual. La segunda opción, más práctica, utiliza técnicas de procesamiento de señales para seleccionar los mismos. 


## 2.4 Clasificar radios

Para realizar la clasificación de los radios se utiliza el algoritmo de aprendizaje automático Random Forest. Este algoritmo requiere que el vector de entrada tenga una longitud fija por lo que es necesario realizar una transformación del perfil de los radios acumulados (longitud variable) a un vector de características de longitud fija. Las características que se utilizan son las siguientes:

* Proporcion entre los valores de intensidad máxima y mínima (se considera que un pixel tiene intensidad máxima si está por encima del 10% de intensidad media dentro del anillo de crecimiento. Es de intensidad baja si está por debajo del 90% de intensidad de la media del radio).
* Varianza de la serie.
* Media de las serie
* Coeficiente  1 de aproximacion por polinomio de orden 1 a la serie
* Coeficiente 2 de aproximacion por polinomio de orden 1 a la serie.



# 3 Software

El código de la aplicación se encuentra en el repositorio GitHub https://github.com/hmarichal93/deteccionMadera

## 3.1 Bibliotecas utilizadas

# 4 Demostración

In [1]:
#########################################
# explicitly calling the HTML renderer:
#########################################
from IPython.display import HTML
HTML("""
<div align="middle">
<video width="80%" controls>
      <source src="./entregable/HaySegmento1.mp4" type="video/mp4">
</video></div>""")

In [2]:
HTML("""
<div align="middle">
<video width="80%" controls>
      <source src="./entregable/noHaySegmento1.mp4" type="video/mp4">
</video></div>""")

In [3]:
HTML("""
<div align="middle">
<video width="80%" controls>
      <source src="./entregable/todosLosPerfiles_hay1.mp4" type="video/mp4">
</video></div>""")

In [4]:
HTML("""
<div align="middle">
<video width="80%" controls>
      <source src="./entregable/rajadura1.mp4" type="video/mp4">
</video></div>""")

# 5 Analisis de Resultados

## 5.1 Selección de características
No se observan mejoras al utilizar la imagen en color
Entrenamiento.

# 5 Contacto

Correo: hmarichal93@gmail.com
